In [1]:
# If you are using Anconda, please re-download this file to current directory:
# https://raw.githubusercontent.com/shawnrhoads/gu-psyc-347/master/course-env.yml
# Then, run this cell
# !conda env update --file course-env.yml

In [2]:
# here we will import all of our packages for the exercise

from scipy.optimize import minimize # finding optimal params in models
from scipy import stats             # statistical tools
import numpy as np                  # matrix/array functions
import pandas as pd                 # loading and manipulating data
import requests                     # downloading data from the internet
import ipywidgets as widgets        # interactive display
import matplotlib.pyplot as plt     # plotting
%matplotlib inline

In [3]:
#solution:

# please load in the data into a pandas DataFrame, 
# then print the first five rows to check that everything loaded correctly
# hint: we've done this in previous exercises
url = 'https://raw.githubusercontent.com/shawnrhoads/gu-psyc-347/master/docs/static/data/OConnell_COVID_MTurk_noPII_post_peerreview.csv'
df = pd.read_csv(url)

display(df.head())

,subID,mturk_randID,suspect_itaysisso,Country,Region,ISP,loc_US,loc_state,loc_zipcode,loc_County,...,education_4yr,STAB_total_centered,STAB_total_min32,silhouette_dist_X_min81,silhouette_dist_X_inches,violated_distancing,STAB_rulebreak_rmECONOMIC,STAB_total_rmECONOMIC,STAB_total_rmECONOMIC_centered,household_income_coded_centered
0,1001,8797,0,United States,CT,"AS7015 Comcast Cable Communications, LLC",Yes,Connecticut,6511,New Haven County,...,0,-3.946565,19,441.0,110.332750,0,9,48,-2.076336,1.269231
1,1002,3756,0,United States,IL,"AS7018 AT&T Services, Inc.",Yes,California,90280,Los Angeles County,...,0,39.053436,62,287.0,71.803856,1,24,88,37.923664,-3.730769
2,1003,3798,0,United States,OH,AS10796 Charter Communications Inc,Yes,Ohio,44883,Seneca County,...,0,40.053436,63,313.0,78.308731,0,23,85,34.923664,-2.730769
3,1004,2965,0,United States,TX,"AS7018 AT&T Services, Inc.",Yes,Texas,77019,Harris County,...,1,-9.946565,13,452.0,113.084820,0,8,42,-8.076336,NaN
4,1005,5953,0,United States,NC,AS20115 Charter Communications,Yes,North Carolina,28334,Sampson County,...,0,-17.946566,5,297.0,74.305733,0,8,34,-16.076336,-2.730769


In [4]:
var_names = ['age_centered',
             'sex_1f',
             'education_4yr',
             'household_income_coded_centered',
             'highrisk_self_or_livewith',
             'ppe_freq_coded',
             'STAB_total_centered',
             'silhouette_dist_X_min81']

df.dropna(subset=var_names,
          inplace=True)

In [5]:
# solution: 

# get number of subjects and create X and y

n_subjects = len(df)

# store columns of interest in a single numpy array 
    # age_centered - age (continuous; 18-65)
    # sex_1f - sex (0: male; 1: female)
    # education_4yr - education (0: < 4-year degree; 1: >= 4-year degree)
    # household_income_coded_centered - household income (continuous)
    # highrisk_self_or_livewith - high-risk for serious illness for self or someone the participant lives with (0: no; 1: yes)
    # ppe_freq_coded_2 - PPE use frequency (continuous; 1: Never, 5: Always)
    # stab_total_centered - antisocial behavior (continuous)
    # silhouette_dist_x_min81 - distance kept from others in past week (continuous)

x0 = np.ones((n_subjects,))
x1 = df['age_centered'].values # age
x2 = df['sex_1f'].values # sex
x3 = df['education_4yr'].values # edu
x4 = df['household_income_coded_centered'].values # income
x5 = df['highrisk_self_or_livewith'].values # high-risk

x6_new = []
for ans in df['ppe_freq_coded']:
    if ans == 'Never':
        x6_new.append(1)
    elif ans == 'Rarely':
        x6_new.append(2)
    elif ans == 'Sometimes':
        x6_new.append(3)
    elif ans == 'Often':
        x6_new.append(4)
    elif ans == 'Always':
        x6_new.append(5)
x6 = np.array(x6_new) # ppe

x7 = df['STAB_total_centered'].values # antisociality (stab)


X = np.stack((x0, x1, x2, x3, x4, x5, x6, x7), axis=1)
y = df['silhouette_dist_X_min81'].values # distance from others

In [6]:
#solution:

# using a for loop, please print the means and standard deviations of these variables

for name, data in zip(var_names, [x1, x2, x3, x4, x5, x6, x7, y]):
    print(f'{name}: mean={np.mean(data):.2f}, sd={np.std(data):.2f}')

age_centered: mean=0.38, sd=10.26
sex_1f: mean=0.42, sd=0.49
education_4yr: mean=0.54, sd=0.50
household_income_coded_centered: mean=0.00, sd=1.79
highrisk_self_or_livewith: mean=0.36, sd=0.48
ppe_freq_coded: mean=2.88, sd=1.62
STAB_total_centered: mean=-4.95, sd=25.17
silhouette_dist_X_min81: mean=390.70, sd=131.67


In [7]:
# define a function called ols() that computes betas for the model
# no need to edit anything here
# see module-02-00_Linear-Modeling for derivation

def ols(X, y):
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y
    return beta_hat

In [8]:
# solution:

# compute the betas
beta_hats = ols(X, y)

In [9]:
#solution:

# print the names of each variable
x_vars = ['Constant', 'Age','Sex','Education',
          'Household Income','High-risk','PPE Use',
          'Antisocial behavior']

for variable, beta_value in zip(x_vars, beta_hats):
    
    print(fr'coefficient for {variable} = {beta_value:.2f}')

coefficient for Constant = 346.50
coefficient for Age = -0.26
coefficient for Sex = 31.11
coefficient for Education = -66.38
coefficient for Household Income = 10.66
coefficient for High-risk = 33.21
coefficient for PPE Use = 16.09
coefficient for Antisocial behavior = -1.76


In [10]:
# interactive display (if not using Jupyter Book)
%config InlineBackend.figure_format = 'retina'

@widgets.interact(mu=widgets.FloatSlider(0.0, min=-2.0, max=2.0),
                  sigma=widgets.FloatSlider(1.0, min=0.5, max=2.0))

def plot_normal_dist(mu=0, sigma=1):

    # Generate pdf & samples from normal distribution with mu/sigma
    rv = stats.norm(mu, sigma)
    x = np.linspace(-5, 5, 100)
    y = rv.pdf(x)
    samples = rv.rvs(1000)

    # Plot
    fig, ax = plt.subplots()
    ax.hist(samples, 20, density=True, color='g', histtype='stepfilled', alpha=0.8,
          label='histogram')
    ax.plot(x, y, color='orange', linewidth=3, label='pdf')
    ax.vlines(mu, 0, rv.pdf(mu), color='red', linewidth=3, label='$\mu$')
    ax.vlines([mu-sigma, mu+sigma], 0, rv.pdf([mu-sigma, mu+sigma]), colors='red',
            color='b', linewidth=3, label='$\sigma$')
    ax.set(xlabel='x', ylabel='probability density', xlim=[-5, 5], ylim=[0, 1.0])
    ax.legend()

interactive(children=(FloatSlider(value=0.0, description='mu', max=2.0, min=-2.0), FloatSlider(value=1.0, desc…

In [11]:
def loglikelihood(beta_hat, X, y):
    """The likelihood function for a linear model with noise sampled from a Gaussian distribution with zero mean and unit variance.

    Args:
    beta_hat (float): Estimates of the slope parameters.
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding measurement values to the inputs.

    Returns:
    ndarray: the likelihood values for the beta_hat estimates
    """
    
    # compute the predicted y based on inputted beta_hats
    # here, dot product is equivalent to the weighted sum of values 
    # (i.e., b0 + b1*x1 + b2*x2 + .. bd*xd)
    predicted_y = np.dot(X, beta_hat) 

    # Compute Gaussian likelihood
    pdf = stats.norm.logpdf(y, loc=predicted_y)
    
    return pdf

In [12]:
#solution:

# please compute the negative loglikelihood in this new function
# note: this function calls the likelihood() function we defined earlier
def minimize_negll(params, X, y):
    """The likelihood function for a model with noise sampled from a
    Gaussian distribution with zero mean and unit variance.

    Args:
    params (list)
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.

    Returns:
    scalar: the neg sum of the loglikelihoods for the beta_hat estimate
    """
    
    # Compute Gaussian loglikelihood
    loglik = loglikelihood(params, X, y)
    
    negLL = -np.sum(loglik)
    
    return negLL

In [13]:
# specify starting points
init_x = (350, -.2, 52, -60, 10, 30, 25, -1)

# minimize MSE for linear function using scipy.optimize.minimize
mle_results = minimize(minimize_negll, # objective function
                       init_x, # estimated starting points
                       args=(X, y), # arguments
                       method = 'Nelder-Mead')

In [14]:
#solution:

# print the names of each variable
x_vars = ['Constant', 'Age','Sex','Education',
          'Household Income','High-risk','PPE Use',
          'Antisocial behavior']

mle_betas = mle_results.x

for variable, beta_value in zip(x_vars, mle_betas):
    
    print(fr'coefficient for {variable} = {beta_value:.2f}')

coefficient for Constant = 346.50
coefficient for Age = -0.26
coefficient for Sex = 31.11
coefficient for Education = -66.38
coefficient for Household Income = 10.66
coefficient for High-risk = 33.22
coefficient for PPE Use = 16.09
coefficient for Antisocial behavior = -1.76


In [15]:
def r_squared(y, predicted_y):
    SS_total = np.sum((y - np.mean(y))**2)
    SS_residual = np.sum((y - predicted_y)**2)
    return 1-(SS_residual/SS_total)

In [16]:
predicted_y = np.dot(X, mle_results.x)
print(f"R^2: {r_squared(y, predicted_y):.3f}")

R^2: 0.248
